In [1]:
import json
import minsearch
import requests
from dotenv import load_dotenv
import os

# Load and prepare documents
documents_path = "documents.json" 
with open(documents_path, 'r', encoding='utf-8') as file:
    documents_raw = json.load(file)

documents = []

for course in documents_raw:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Refit the index with the expanded document set
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

def search(query, filter_course=None):
    boost = {'question': 3.0, 'section': 0.5}
    
    filter_dict = {}
    if filter_course:
        filter_dict = {'course': filter_course}
        
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=5
    )
    
    return results

def build_prompt(query, diagnosis_result, search_results):
    prompt_template = """
You are a dermatology AI assistant for the Acne Sense app. Your role is to provide helpful, accurate information about acne based on the user's skin analysis and questions.

SKIN ANALYSIS RESULT:
{diagnosis}

USER QUESTION: {question}

CONTEXT FROM KNOWLEDGE BASE:
{context}

Provide a helpful, compassionate response that:
1. Acknowledges their specific acne condition
2. Offers relevant treatment suggestions and skincare advice based on their acne type
3. Clearly indicates when professional dermatologist consultation is recommended
4. Uses only information from the provided context
5. Avoids making definitive medical diagnoses or prescribing specific medications
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(diagnosis=diagnosis_result, question=query, context=context).strip()
    return prompt

def llm(prompt):
    """
    Send a prompt to the Ollama API running locally with Qwen 2.5 7B model
    """
    # Default Ollama API endpoint - modify if using a different host/port
    url = "http://localhost:11434/api/generate"
    
    # Request payload for Ollama's API
    payload = {
        "model": "qwen2:7b",  # Use qwen2:7b model
        "prompt": prompt,
        "stream": False,      # Don't stream the response
        "options": {
            "temperature": 0.2,
            "top_p": 0.95,
            "top_k": 40,
            "max_tokens": 800
        }
    }
    
    try:
        # Send the request to Ollama
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise exception for non-2xx responses
        
        # Extract the generated text from the response
        response_data = response.json()
        return response_data["response"]
    
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API: {e}")
        return f"Error generating response: {str(e)}"

def rag(query, diagnosis_result=None, filter_course=None):
    search_results = search(query, filter_course)
    prompt = build_prompt(query, diagnosis_result, search_results)
    answer = llm(prompt)
    return answer

# Test the basic LLM function
response = llm('write that this is a test')
print(response)

# Example: Test with a question about the app
app_query = "How does the Acne Sense app protect my privacy?"
app_answer = rag(app_query, filter_course="acne-sense-app-info")
print("Question about the app:")
print(f"Q: {app_query}")
print(f"A: {app_answer}")
print("\n" + "="*50 + "\n")

This is indeed a test.
Question about the app:
Q: How does the Acne Sense app protect my privacy?
A: Hello! I'm glad you're using Acne Sense to better understand your skin condition and find suitable treatment options. Based on our privacy guidelines, it's important to know that Acne Sense is designed with your privacy in mind. Facial analysis happens either directly on your device when possible or securely through encrypted cloud processing, ensuring that your images are not permanently stored after the analysis process.

To help you track your progress and monitor improvements over time, use the 'Progress' feature within the app. Take new photos consistently, ideally every 2-4 weeks under similar lighting conditions and angles to see how your skin evolves.

Regarding your specific acne condition, Acne Sense analyzes different types of lesions such as papules, pustules, nodules, and cysts. The AI-powered assessment provides a personalized evaluation of your acne severity and suggests 